<p><img style="padding: 0 15px; float: left;" src="Images/FTLogo300.jpg" alt="FT Crusader Logo" Width='140' Height= '250'/></p> 
<p> <h2> Frogtown DataLoad Workbook, 05/06/18</h2> <a name="tc"></a>
<p> <h3> By Frogtown Crusader (Abu Nayeem)</h3>

This is just clean coded process going over the uploading steps assuming all the preliminary steps after decoder are made


## Table of contents <a name="content"></a>
* [Data Setup](#setup)
* [Create Variables](#create)
* [Intersection Table: Preparation](#intersection)
* [Address Table: Preparation](#address)
* [Preliminary Data Cleaning](#pre)


### Data Setup <a name="setup"></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import HTML
from IPython.display import display
import requests # library to handle requests

#Load Data
#df_crime = pd.read_csv('Datasets/Crime_Incident_Report_-_Dataset.csv')

#rename columns
#cols= ['Case','Date','Time','Code','IncType','Incident','Grid','NNum','Neighborhood','Block','CallDispCode','CallDisposition', 'Count']
#df_crime.columns= cols

#selection for Frogtown and nearby area
#df=df_crime.query('Grid in [66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110]')

#Socarata upload Method
from sodapy import Socrata

#New Upload Method Get Information from Socrata API
client = Socrata("information.stpaul.gov", None)

#Easier to bulk upload
results = client.get("gppb-g9cg", limit=1000000)
df = pd.DataFrame.from_records(results)
# Find Max Date Value
results = client.get("gppb-g9cg", limit=1)
r_max = pd.DataFrame.from_records(results)

#rename columns [Note the order of Columns have changed]
cols= ['Block','CallDispCode','CallDisposition','Case','Code', 'Count','Date','Grid','Incident','IncType','Neighborhood','NNum','Time']
df.columns= cols
df=df.dropna()
df = df.astype({"Case": int, "Code": int, "Grid":float, "NNum":int,"Count":int})
#select respective Grids of interest
df=df.query("Grid in (66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110)")

df.head(4)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,Neighborhood,NNum,Time
4,MACKUBIN ST & THOMAS,A,Advised,20261220,9954,1,2020-12-09T23:50:55.000,89.0,Proactive Police Visit,Proactive Police Visit,7 - Thomas/Dale(Frogtown),7,2020-12-09T23:50:55.000
7,11X EMPIRE DR,A,Advised,20261214,9954,1,2020-12-09T23:40:01.000,91.0,Proactive Police Visit,Proactive Police Visit,7 - Thomas/Dale(Frogtown),7,2020-12-09T23:40:01.000
8,75X JACKSON ST,A,Advised,20261213,9954,1,2020-12-09T23:38:42.000,91.0,Proactive Police Visit,Proactive Police Visit,6 - North End,6,2020-12-09T23:38:42.000
19,45X UNIVERSITY AV W,A,Advised,20261195,9954,1,2020-12-09T23:22:44.000,89.0,Proactive Police Visit,Proactive Police Visit,7 - Thomas/Dale(Frogtown),7,2020-12-09T23:22:44.000


### Create New Variables <a name="create"></a>

In [2]:
#Add Time Variables
df= df[df.Case != 18254093] #messed up time variable

#Convert Date to Datetime!
from datetime import datetime

df['DateTime']= pd.to_datetime(df['Date']) # Create new column called DateTime
df['Year']= df['DateTime'].dt.year #create year column
df['DayofWeek']=df['DateTime'].dt.dayofweek #create day of the week column where default 0=Monday
df['Weekend'] = df['DayofWeek'].apply(lambda x: 1 if (x>4)  else 0) #Create a weekend category
df['Month'] = df['DateTime'].dt.month # Create Month Category
df['Day'] = df['DateTime'].dt.day #Create Day of the Current month
df['DayYear'] = df['DateTime'].dt.dayofyear  #Create Day of the year (0-365)
df['Day_Max'] = df.iloc[0,-1] #selects uptodate day; NOTE: the data is sorted chronologically

#Hour Data
df['TimeHour']= pd.to_datetime(df['Time'])
df['Hour'] = df['TimeHour'].dt.hour.astype(int) #Create Hour Colum
df['LateNight'] = df['Hour'].apply(lambda x: 1 if (x>21 or x<5)  else 0) #Latenight designation from 10Pm to 6PM

#Creating the intersection Column. Note: the Block column has the address information
df.Block = df.Block.astype(str) #first change the type to string
df['Block']= df['Block'].str.lower() #lowercase string to create uniformity

#While scanning the data I noticed that all intersections had "&" 
df['Intersection'] = df['Block'].apply(lambda x: 1 if '&' in x else 0) #intersection

df.head(5)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,DayofWeek,Weekend,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection
4,mackubin st & thomas,A,Advised,20261220,9954,1,2020-12-09T23:50:55.000,89.0,Proactive Police Visit,Proactive Police Visit,...,2,0,12,9,344,344,2020-12-09 23:50:55,23,1,1
7,11x empire dr,A,Advised,20261214,9954,1,2020-12-09T23:40:01.000,91.0,Proactive Police Visit,Proactive Police Visit,...,2,0,12,9,344,344,2020-12-09 23:40:01,23,1,0
8,75x jackson st,A,Advised,20261213,9954,1,2020-12-09T23:38:42.000,91.0,Proactive Police Visit,Proactive Police Visit,...,2,0,12,9,344,344,2020-12-09 23:38:42,23,1,0
19,45x university av w,A,Advised,20261195,9954,1,2020-12-09T23:22:44.000,89.0,Proactive Police Visit,Proactive Police Visit,...,2,0,12,9,344,344,2020-12-09 23:22:44,23,1,0
23,17x charles av,A,Advised,20261188,1400,1,2020-12-09T23:09:22.000,90.0,Vandalism,Criminal Damage to Property,...,2,0,12,9,344,344,2020-12-09 23:09:22,23,1,0


### Prepare Intersection table <a name="intersection"></a>

In [3]:
#Load clean intersections key
df_Intersection_key = pd.read_csv('Datasets/Intersection_key_clean.csv')

# Create a new dateframe specifying only intersections from primary dataset
dfI=df.query('Intersection ==1')
print('The intersection table dimension are ' + str(dfI.shape))
#print(dfI.Block.head(10))


#Split the strings
new=dfI['Block'].str.split("& ", n = 1, expand = True) 
dfI['Inter2']= new[1]
new=dfI['Block'].str.split(" ", n = 1, expand = True) #Note the code specifies the first time a space occured
dfI['Inter1']=new[0]

#Create the IndexKey; recall we prepared the IntersectionKey where it considers any order
dfI['IndexKey']= dfI['Inter1']+ '_' + dfI['Inter2']
dfI.reset_index()
dfI=pd.merge(dfI, df_Intersection_key, on='IndexKey', how='left')



The intersection table dimension are (10909, 25)


In [4]:
#Drop rows with missing coordinates
dfI=dfI[dfI['Coordinates'].notnull()]

# Separate Latitude and Longitude 
new=dfI['Coordinates'].str.split(",", n = 1, expand = True) 
# making seperate first name column from new data frame 
dfI['Latitude']= pd.to_numeric(new[0]) #pd.to_numeric convert it to float
dfI['Longitude']= pd.to_numeric(new[1])

#Renaming columns

dfI['Block']=dfI['OutputKey'] #for practical purposes it makes sense
Drop_col=['Inter2','Inter1', 'IndexKey', 'Coordinates', 'OutputKey']
dfI_Final=dfI.drop(Drop_col, axis=1,)
dfI_Final.head(5)


,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection,Latitude,Longitude
0,mackubin_thomas,A,Advised,20261220,9954,1,2020-12-09T23:50:55.000,89.0,Proactive Police Visit,Proactive Police Visit,...,12,9,344,344,2020-12-09 23:50:55,23,1,1,44.959365,-93.121234
1,dale_thomas,A,Advised,20261176,9954,1,2020-12-09T22:46:33.000,89.0,Proactive Police Visit,Proactive Police Visit,...,12,9,344,344,2020-12-09 22:46:33,22,1,1,44.959342,-93.126371
2,western_university,G,Gone on Arrival,20261115,1400,1,2020-12-09T20:50:18.000,90.0,Vandalism,Criminal Damage to Property,...,12,9,344,344,2020-12-09 20:50:18,20,0,1,44.955870,-93.116148
3,marshall_dunlap,A,Advised,20261069,9954,1,2020-12-09T19:38:10.000,106.0,Proactive Police Visit,Proactive Police Visit,...,12,9,344,344,2020-12-09 19:38:10,19,0,1,44.948539,-93.149184
4,arundel_edmund,A,Advised,20261047,9954,1,2020-12-09T19:01:55.000,89.0,Proactive Police Visit,Proactive Police Visit,...,12,9,344,344,2020-12-09 19:01:55,19,0,1,44.958472,-93.118665


### Prepare Address Decoder Table <a name="address"></a>

It is so remarkably short

In [5]:
#Split Data
dfW=df.query('Intersection==0')

#Load Complete Decoder Key
df_C= pd.read_csv('Datasets/SemiKey.csv')
df_C= df_C[['Block','Latitude','Longitude']]

# Merge with the dataset and remove missing values
dC=pd.merge(dfW, df_C, on='Block', how='left')
dC=dC.fillna('Mi')
dC=dC.query('Latitude != "Mi"')

#Bringing the data back together 
fg= dfI.append(dC, ignore_index=True)

### Preliminary Pre-Cleaning steps <a name="clean"></a>

It is good to do the data pre-cleaning steps here to reduce clutter on a working notebook. A few edits include renaming some values, clustering crimes together and creating some dummies. It is saved in a csv, which is used for execution

In [6]:
#Few Quick Edits
fg.CallDisposition.loc[(fg['CallDisposition'] == 'G - Gone on Arrival')] = 'Gone on Arrival'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'A - Advised')] = 'Advised'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'RR - Report Written')] = 'Report Written'
fg.Incident.loc[(fg['Incident'] == 'Simple Asasult Dom.')] = 'Simple Assault Dom.'
fg.Incident.loc[(fg['Incident'] == 'Graffiti')] = 'Vandalism'
fg.Incident.loc[fg["Incident"].isin([ "Rape","Agg. Assault",'Homicide'])]= 'Violent'
fg.Incident.loc[fg["Incident"].isin(["Simple Assault Dom.","Agg. Assault Dom."])]= 'Domestic Assault'

#[fg["Incident"].isin(["Simple Assault Dom.", "Rape"])


#Create a dummy for each crime category
fg= pd.concat([fg,pd.get_dummies(fg['Incident'])], axis=1)
fg= pd.concat([fg,pd.get_dummies(fg['CallDisposition'])], axis=1)

fg.to_csv('Datasets/FGCrime_Final.csv', encoding='utf-8', index=False)

In [8]:
fg.query('Year==2020')

,Block,CallDispCode,CallDisposition,Case,Code,Coordinates,Count,Date,DateTime,Day,...,Vandalism,Violent,10 - Como,11 - Hamline/Midway,13 - Union Park,7 - Thomas/Dale(Frogtown),8 - Summit/University,Advised,Gone on Arrival,Report Written
0,arundel_university,A,Advised,20140593,9954,"44.955853, -93.118679",1,2020-07-15T17:34:12.000,2020-07-15 17:34:12,15,...,0,0,0,0,0,0,0,1,0,0
1,lexington_fuller,RR,Report Written,20140558,411,"44.953930, -93.146544",1,2020-07-15T16:47:00.000,2020-07-15 16:47:00,15,...,0,0,0,0,0,0,0,0,0,1
2,rice_university,A,Advised,20140524,9954,"44.955847, -93.105959",1,2020-07-15T16:14:30.000,2020-07-15 16:14:30,15,...,0,0,0,0,0,0,0,1,0,0
3,rice_charles,G,Gone on Arrival,20140498,1800,"44.957614, -93.105951",1,2020-07-15T15:20:06.000,2020-07-15 15:20:06,15,...,0,0,0,0,0,0,0,0,1,0
4,grotto_blair,A,Advised,20140435,9954,"44.961170, -93.131442",1,2020-07-15T13:31:25.000,2020-07-15 13:31:25,15,...,0,0,0,0,0,0,0,1,0,0
5,arundel_vanburen,A,Advised,20140428,9954,"44.962067, -93.118676",1,2020-07-15T13:23:38.000,2020-07-15 13:23:38,15,...,0,0,0,0,0,0,0,1,0,0
6,arundel_university,A,Advised,20140412,9954,"44.955853, -93.118679",1,2020-07-15T13:03:38.000,2020-07-15 13:03:38,15,...,0,0,0,0,0,0,0,1,0,0
7,rice_pennsylvania,RR,Report Written,20140423,314,"44.960366, -93.105941",1,2020-07-15T12:20:00.000,2020-07-15 12:20:00,15,...,0,0,0,0,0,0,0,0,0,1
8,chatsworth_vanburen,RR,Report Written,20804139,641,"44.962081, -93.141554",1,2020-07-15T10:20:00.000,2020-07-15 10:20:00,15,...,0,0,0,0,0,0,0,0,0,1
9,rice_charles,G,Gone on Arrival,20140158,1800,"44.957614, -93.105951",1,2020-07-15T04:48:23.000,2020-07-15 04:48:23,15,...,0,0,0,0,0,0,0,0,1,0


In [7]:
Ward1.query('ElectionDescription in ("11/08/2016 - STATE GENERAL", "08/09/2016 - STATE PRIMARY","11/07/2017 - MUNICIPAL GENERAL","11/07/2017 - SCHOOL DISTRICT GENERAL"  ,"11/08/2016 - STATE GENERAL","08/09/2016 - STATE PRIMARY","11/03/2015 - MUNICIPAL GENERAL","11/03/2015 - SCHOOL DISTRICT GENERAL","11/08/2011 - MUNICIPAL GENERAL","11/06/2018 - STATE GENERAL","08/14/2018 - STATE PRIMARY")' 

SyntaxError: unexpected EOF while parsing (<ipython-input-7-b3cfb0eeffb8>, line 1)